<a href="https://colab.research.google.com/github/sightes/BIGDATA-UBO2022/blob/main/Pyspark_Twitter_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Lectura Twitter Offline

In [ ]:
import configparser
import tweepy
import pandas as pd
# Set up your credentials
consumer_key='RT7J7AamjjqQcygpb3spKSPGX'
consumer_secret='g5hloTxWrzCFxIyzZ4GGnpRcZ2nNrfWRvbA20RVMqd4TRSvYVX'
access_token ='27903879-2fgC78QL20NzCeYYxKWOmjXLx81I8RkXWona5n5v1'
access_secret='kQoXqHLacDzxcaYjcqZ4YatPZElT5G5mMDtjyRzVYtrro'
config=configparser.ConfigParser()
config.read('config.ini')
auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_secret)
api=tweepy.API(auth)
tweets=api.home_timeline()
data=[]
for tweet in tweets:
   data.append([tweet.created_at,tweet.user.screen_name
                ,tweet.text,tweet.entities.get('hashtags')])
pd.DataFrame(data,columns=['fecha','usuario','mensaje','hashtag'])

###Streaming Online

In [21]:
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget 'https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz' -O 'spark-3.3.1-bin-hadoop3.tgz'
!tar -xvf /content/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark 


In [22]:
%%python --bg
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json
# Set up your credentials
consumer_key='RT7J7AamjjqQcygpb3spKSPGX'
consumer_secret='g5hloTxWrzCFxIyzZ4GGnpRcZ2nNrfWRvbA20RVMqd4TRSvYVX'
access_token ='27903879-2fgC78QL20NzCeYYxKWOmjXLx81I8RkXWona5n5v1'
access_secret='kQoXqHLacDzxcaYjcqZ4YatPZElT5G5mMDtjyRzVYtrro'

class TweetsListener(StreamListener):

  def __init__(self, csocket):
      self.client_socket = csocket

  def on_data(self, data):
      try:
          msg = json.loads( data )
          print( msg['text'].encode('utf-8') )
          self.client_socket.send( msg['text'].encode('utf-8') )
          return True
      except BaseException as e:
          print("Error on_data: %s" % str(e))
      return True

  def on_error(self, status):
      print(status)
      return True

def sendData(c_socket):
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)
  twitter_stream = Stream(auth, TweetsListener(c_socket),tweet_mode='extended')
  twitter_stream.filter(track=["Mall"])

if __name__ == "__main__":
  s = socket.socket()         # Create a socket object
  host = "127.0.0.1"     # Get local machine name
  port = 5554                 # Reserve a port for your service.
  s.bind((host, port))        # Bind to the port
  print("Listening on port: %s" % str(port))
  s.listen(5)                 # Now wait for client connection.
  c, addr = s.accept()        # Establish connection with client.
  print( "Received request from: " + str( addr ) )
  sendData( c )

In [23]:
import os 
os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME']='/content/spark-3.3.1-bin-hadoop3'
import findspark 
findspark.init()

In [33]:

sc.stop()

In [34]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

sc=SparkContext()
ssc=StreamingContext(sc,10)
sqlContext=SQLContext(sc)

socket_stream=ssc.socketTextStream('127.0.0.1',5554)
lines=socket_stream.window(20)

from collections import namedtuple

fields =('tag','count')
Tweet=namedtuple('Tweet',fields)

(lines.flatMap(lambda text: text.split(''))
.filter(lambda word:word.lower().startswith('#'))
.map(lambda word:(word.lower(),1))
.reduceByKey(lambda a,b:a+b)
.map(lambda rec:Tweet(rec[0],rec[1])) 
.foreachRDD(lambda rdd: rdd.toDF().sort(desc('count'))
  .limit(10).registerTempTable('tweets')))

In [ ]:
ssc.start()